In [2]:
import FinanceDataReader as fdr
import pandas as pd

# 원래 티커 리스트 (숫자형)
tickers = [
    '136660', '356860', '219130', '031980', '464500', '212710', '036930', '092070',
    '241790', '417500', '045300', '089970', '319660', '429270', '311320', '451220',
    '064520', '054620', '096610', '149010', '172670', '222160', '272110', '122640',
    '101400', '011930', '087600', '054940', '030530', '183300', '227950', '241770',
    '102710', '083310', '425420', '003160', '005935', '079370', '445180', '059090',
    '082270', '005930', '029460', '226590', '140070', '112290', '061970', '084370',
    '067310', '264660', '330860', '142210', '101160', '217500', '086390', '094170',
    '114810', '131970', '102120', '080220', '052900', '382800', '195870', '160980',
    '394280', '389020', '064760', '058470', '232140', '033160', '425040', '083450',
    '093520', '053610', '166090', '101490', '240810', '420770', '405100', '033640',
    '348350', '396270', '005290', '071280', '098120', '169670', '080520', '365590',
    '445090', '032580', '000660', '036540', '036200', '171010', '117670', '054450',
    '089890', '042700', '147760', '322310', '200710', '089790', '144960', '402490',
    '432720', '094360', '253590', '077500', '306620', '097800', '036810', '077360',
    '252990', '006200', '424980', '396470', '041520', '066310', '039440', '217190',
    '105330', '093640', '418420', '170920', '095610', '348210', '357780', '417840',
    '200470', '403870', '025560', '232680', '000990', '078350', '131290', '399720',
    '482630', '119830', '489500', '146320', '074600', '095340', '039030', '089030',
    '080580', '440110', '254490', '064290', '092220', '323350', '059120', '281820',
    '412350', '452430', '052860', '024850', '033170', '355150', '092870', '452160',
    '317330', '303030', '413300'
]

# 최종 수집할 데이터프레임
final_data = pd.DataFrame()

# 실패한 티커 기록
failed_tickers = []

# 데이터 수집
for ticker in tickers:
    try:
        df = fdr.DataReader(ticker, start='2020-01-01', end='2024-12-31')[['Close']]
        df.rename(columns={'Close': str(ticker)}, inplace=True)
        final_data = pd.concat([final_data, df[str(ticker)]], axis=1)
        print(f"{ticker} downloaded successfully")
    except Exception as e:
        print(f"Failed to download {ticker}: {e}")
        failed_tickers.append(ticker)

# 결측치 선형보간
final_data = final_data.interpolate(method='linear')

# 파일로 저장
final_data.to_csv('korean_stocks_close.csv')

# 실패한 티커 저장
if failed_tickers:
    pd.Series(failed_tickers).to_csv('failed_tickers.csv', index=False)
    print(f"Some tickers failed: {failed_tickers}")
else:
    print("All tickers downloaded successfully!")


136660 downloaded successfully
356860 downloaded successfully
219130 downloaded successfully
031980 downloaded successfully
464500 downloaded successfully
212710 downloaded successfully
036930 downloaded successfully
092070 downloaded successfully
241790 downloaded successfully
417500 downloaded successfully
045300 downloaded successfully
089970 downloaded successfully
319660 downloaded successfully
429270 downloaded successfully
311320 downloaded successfully
451220 downloaded successfully
064520 downloaded successfully
054620 downloaded successfully
096610 downloaded successfully
149010 downloaded successfully
172670 downloaded successfully
222160 downloaded successfully
272110 downloaded successfully
122640 downloaded successfully
101400 downloaded successfully
011930 downloaded successfully
087600 downloaded successfully
054940 downloaded successfully
030530 downloaded successfully
183300 downloaded successfully
227950 downloaded successfully
241770 downloaded successfully
102710 d

In [3]:
stocks = pd.read_csv('korean_stocks_close.csv')
stocks

,Unnamed: 0,136660,356860,219130,031980,464500,212710,036930,092070,241790,...,452430,052860,024850,033170,355150,092870,452160,317330,303030,413300
0,2020-01-02 00:00:00,3035,NaN,5600,8630,NaN,NaN,8160,10100,10400,...,NaN,5290,1120,1155,NaN,6632,NaN,19250,2900,NaN
1,2020-01-03 00:00:00,3260,NaN,5580,8340,NaN,NaN,7930,10250,10350,...,NaN,5210,1085,1150,NaN,7077,NaN,18800,2850,NaN
2,2020-01-06 00:00:00,3145,NaN,5540,8320,NaN,NaN,7920,10000,10150,...,NaN,5180,1090,1140,NaN,7664,NaN,18500,2760,NaN
3,2020-01-07 00:00:00,3195,NaN,5510,8170,NaN,NaN,7910,10100,10100,...,NaN,5000,1105,1160,NaN,7683,NaN,18150,2750,NaN
4,2020-01-08 00:00:00,3100,NaN,5310,8250,NaN,NaN,7640,10200,10000,...,NaN,4660,1010,1105,NaN,7693,NaN,17150,2550,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2024-12-23 00:00:00,1400,13260.0,13470,32950,4135.0,NaN,29650,8660,4815,...,15100.0,1885,2240,942,5050.0,10710,4530.0,31100,872,520.0
1227,2024-12-24 00:00:00,1489,13100.0,13030,32350,3805.0,NaN,30350,8650,4770,...,14690.0,1810,2230,910,5010.0,10350,4590.0,30650,1133,520.0
1228,2024-12-26 00:00:00,1499,12720.0,12780,35000,3665.0,NaN,29700,8520,4740,...,14270.0,1747,2250,888,4845.0,10340,4560.0,29750,1160,580.0
1229,2024-12-27 00:00:00,1389,12940.0,12670,38350,3750.0,NaN,28750,8690,4605,...,14250.0,1738,2305,863,4720.0,10710,4735.0,28800,1159,650.0


In [5]:
# NaN 값이 하나라도 있는 열만 선택
nan_columns = stocks.columns[stocks.isnull().any()]

# 그 열들에 대해 NaN 개수 출력
nan_counts = stocks[nan_columns].isnull().sum()

# NaN이 존재하는 열의 개수
nan_columns_count = len(nan_columns)

print(nan_counts)
print(f"NaN이 있는 열의 개수: {nan_columns_count}")


356860     236
464500    1164
212710    1231
417500     702
429270     888
311320     460
451220     930
172670     883
425420     711
445180     936
226590    1231
112290     626
330860     217
382800     451
394280     675
389020     785
425040     755
420770     839
405100     700
348350     205
396270     617
169670     510
365590     241
445090     954
322310     283
402490     963
432720     943
252990     341
424980     821
396470     942
418420     787
348210     190
357780     149
417840     697
403870     627
399720     589
482630    1231
489500    1231
146320     535
440110     890
254490     759
412350     612
452430    1019
355150     179
452160     963
413300     676
dtype: int64
NaN이 있는 열의 개수: 46


In [9]:
# NaN 있는 열 리스트 만들기
nan_columns_list = stocks.columns[stocks.isnull().any()].tolist()

# NaN 있는 열을 드랍(drop)해서 새로운 DataFrame 만들기
stocks_no_nan = stocks.drop(columns=nan_columns_list)

# 결과 확인
print(stocks_no_nan.shape)
stocks_no_nan

(1231, 118)


,Unnamed: 0,136660,219130,031980,036930,092070,241790,045300,089970,319660,...,092220,323350,059120,281820,052860,024850,033170,092870,317330,303030
0,2020-01-02 00:00:00,3035,5600,8630,8160,10100,10400,3826,7560,11725,...,986,16000,5146,23050,5290,1120,1155,6632,19250,2900
1,2020-01-03 00:00:00,3260,5580,8340,7930,10250,10350,3762,7740,11450,...,986,16000,5307,23050,5210,1085,1150,7077,18800,2850
2,2020-01-06 00:00:00,3145,5540,8320,7920,10000,10150,3735,7700,11775,...,949,16000,5563,23450,5180,1090,1140,7664,18500,2760
3,2020-01-07 00:00:00,3195,5510,8170,7910,10100,10100,3762,7730,11625,...,963,16000,5765,22500,5000,1105,1160,7683,18150,2750
4,2020-01-08 00:00:00,3100,5310,8250,7640,10200,10000,3803,7740,12000,...,926,16000,5644,22600,4660,1010,1105,7693,17150,2550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2024-12-23 00:00:00,1400,13470,32950,29650,8660,4815,2549,7110,17020,...,813,4950,6080,27050,1885,2240,942,10710,31100,872
1227,2024-12-24 00:00:00,1489,13030,32350,30350,8650,4770,2544,6770,17000,...,803,5010,6240,26900,1810,2230,910,10350,30650,1133
1228,2024-12-26 00:00:00,1499,12780,35000,29700,8520,4740,2510,6840,16490,...,789,4975,6210,25700,1747,2250,888,10340,29750,1160
1229,2024-12-27 00:00:00,1389,12670,38350,28750,8690,4605,2475,6840,16500,...,777,5010,6140,26550,1738,2305,863,10710,28800,1159


In [11]:
# 첫 번째 열 이름을 'Date'로 변경
stocks_no_nan = stocks_no_nan.rename(columns={stocks_no_nan.columns[0]: 'Date'})

# 'Date' 열을 datetime 형식으로 변환
stocks_no_nan['Date'] = pd.to_datetime(stocks_no_nan['Date'])

# 날짜를 yyyy-mm-dd 형식으로 포맷 (필요하면)
stocks_no_nan['Date'] = stocks_no_nan['Date'].dt.strftime('%Y-%m-%d')

# 결과 확인
stocks_no_nan


,Date,136660,219130,031980,036930,092070,241790,045300,089970,319660,...,092220,323350,059120,281820,052860,024850,033170,092870,317330,303030
0,2020-01-02,3035,5600,8630,8160,10100,10400,3826,7560,11725,...,986,16000,5146,23050,5290,1120,1155,6632,19250,2900
1,2020-01-03,3260,5580,8340,7930,10250,10350,3762,7740,11450,...,986,16000,5307,23050,5210,1085,1150,7077,18800,2850
2,2020-01-06,3145,5540,8320,7920,10000,10150,3735,7700,11775,...,949,16000,5563,23450,5180,1090,1140,7664,18500,2760
3,2020-01-07,3195,5510,8170,7910,10100,10100,3762,7730,11625,...,963,16000,5765,22500,5000,1105,1160,7683,18150,2750
4,2020-01-08,3100,5310,8250,7640,10200,10000,3803,7740,12000,...,926,16000,5644,22600,4660,1010,1105,7693,17150,2550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2024-12-23,1400,13470,32950,29650,8660,4815,2549,7110,17020,...,813,4950,6080,27050,1885,2240,942,10710,31100,872
1227,2024-12-24,1489,13030,32350,30350,8650,4770,2544,6770,17000,...,803,5010,6240,26900,1810,2230,910,10350,30650,1133
1228,2024-12-26,1499,12780,35000,29700,8520,4740,2510,6840,16490,...,789,4975,6210,25700,1747,2250,888,10340,29750,1160
1229,2024-12-27,1389,12670,38350,28750,8690,4605,2475,6840,16500,...,777,5010,6140,26550,1738,2305,863,10710,28800,1159


In [12]:
stocks_no_nan.to_csv('korean_stocks_close_no_nan.csv', index = False)

In [13]:
nan_columns_list

['356860',
 '464500',
 '212710',
 '417500',
 '429270',
 '311320',
 '451220',
 '172670',
 '425420',
 '445180',
 '226590',
 '112290',
 '330860',
 '382800',
 '394280',
 '389020',
 '425040',
 '420770',
 '405100',
 '348350',
 '396270',
 '169670',
 '365590',
 '445090',
 '322310',
 '402490',
 '432720',
 '252990',
 '424980',
 '396470',
 '418420',
 '348210',
 '357780',
 '417840',
 '403870',
 '399720',
 '482630',
 '489500',
 '146320',
 '440110',
 '254490',
 '412350',
 '452430',
 '355150',
 '452160',
 '413300']

In [15]:
import FinanceDataReader as fdr

# 네가 준 티커 리스트
target_tickers = nan_columns_list

# 전체 한국 상장 종목 불러오기
krx = fdr.StockListing('KRX')

# 티커-종목명 매핑
ticker_name_dict = {}

for ticker in target_tickers:
    name = krx.loc[krx['Code'] == ticker, 'Name']
    if not name.empty:
        ticker_name_dict[ticker] = name.values[0]
    else:
        ticker_name_dict[ticker] = None  # 못 찾은 경우 None 처리

# 결과 확인
print(ticker_name_dict)


{'356860': '티엘비', '464500': '아이언디바이스', '212710': '아이에스티이', '417500': '제이아이테크', '429270': '시지트로닉스', '311320': '지오엘리먼트', '451220': '아이엠티', '172670': '에이엘티', '425420': '티에프이', '445180': '퓨릿', '226590': '엠디바이스', '112290': '와이씨켐', '330860': '네패스아크', '382800': '지앤비에스 에코', '394280': '오픈엣지테크놀로지', '389020': '자람테크놀로지', '425040': '티이엠씨', '420770': '기가비스', '405100': '큐알티', '348350': '위드텍', '396270': '넥스트칩', '169670': '코스텍시스템', '365590': '하이딥', '445090': '에이직랜드', '322310': '오로스테크놀로지', '402490': '그린리소스', '432720': '퀄리타스반도체', '252990': '샘씨엔에스', '424980': '마이크로투나노', '396470': '워트', '418420': '라온텍', '348210': '넥스틴', '357780': '솔브레인', '417840': '저스템', '403870': 'HPSP', '399720': '가온칩스', '482630': '삼양엔씨켐', '489500': '엘케이켐', '146320': '비씨엔씨', '440110': '파두', '254490': '미래반도체', '412350': '레이저쎌', '452430': '사피엔반도체', '355150': '코스텍시스', '452160': '제이엔비', '413300': '티엘엔지니어링'}


In [17]:
print(ticker_name_dict.values())

dict_values(['티엘비', '아이언디바이스', '아이에스티이', '제이아이테크', '시지트로닉스', '지오엘리먼트', '아이엠티', '에이엘티', '티에프이', '퓨릿', '엠디바이스', '와이씨켐', '네패스아크', '지앤비에스 에코', '오픈엣지테크놀로지', '자람테크놀로지', '티이엠씨', '기가비스', '큐알티', '위드텍', '넥스트칩', '코스텍시스템', '하이딥', '에이직랜드', '오로스테크놀로지', '그린리소스', '퀄리타스반도체', '샘씨엔에스', '마이크로투나노', '워트', '라온텍', '넥스틴', '솔브레인', '저스템', 'HPSP', '가온칩스', '삼양엔씨켐', '엘케이켐', '비씨엔씨', '파두', '미래반도체', '레이저쎌', '사피엔반도체', '코스텍시스', '제이엔비', '티엘엔지니어링'])


In [18]:
# 추출할 날짜 리스트
target_dates = ['2020-12-30', '2021-12-30', '2022-12-29', '2023-12-28', '2024-12-30']

# 'Date' 열이 있는지 확인 후, Date를 문자열로 맞춰줌
if not pd.api.types.is_string_dtype(stocks_no_nan['Date']):
    stocks_no_nan['Date'] = stocks_no_nan['Date'].astype(str)

# 해당 날짜에 해당하는 행만 추출
stocks_selected = stocks_no_nan[stocks_no_nan['Date'].isin(target_dates)].reset_index(drop=True)

# 결과 확인
stocks_selected

,Date,136660,219130,031980,036930,092070,241790,045300,089970,319660,...,092220,323350,059120,281820,052860,024850,033170,092870,317330,303030
0,2020-12-30,5040,15000,8300,8210,21200,18100,4401,19900,20275,...,2730,19250,6060,30750,4300,1415,853,15786,17800,2495
1,2021-12-30,3200,20900,17900,21000,22850,16600,5129,22150,26400,...,3451,79900,11950,24050,3970,1260,2520,14447,30850,3220
2,2022-12-29,2800,20050,6850,10600,13200,13300,3760,10600,15400,...,1819,54000,7790,14800,2495,948,1030,8696,13500,1340
3,2023-12-28,1300,19990,26950,34200,23950,12860,3993,14100,21050,...,1557,10000,10170,26800,2745,3120,1236,17221,18690,1594
4,2024-12-30,1399,12780,38050,29450,8820,4630,2520,6870,16500,...,780,5000,6580,26050,1799,2510,864,10600,29650,1125


In [19]:
# 예시로 nan_columns_list 가 있다고 가정
# nan_columns_list = ['356860', '464500', '212710', ...] 

# nan_columns_list에 있는 것들 제외하고 새로운 리스트 만들기
filtered_tickers = [ticker for ticker in tickers if ticker not in nan_columns_list]

print(filtered_tickers)


['136660', '219130', '031980', '036930', '092070', '241790', '045300', '089970', '319660', '064520', '054620', '096610', '149010', '222160', '272110', '122640', '101400', '011930', '087600', '054940', '030530', '183300', '227950', '241770', '102710', '083310', '003160', '005935', '079370', '059090', '082270', '005930', '029460', '140070', '061970', '084370', '067310', '264660', '142210', '101160', '217500', '086390', '094170', '114810', '131970', '102120', '080220', '052900', '195870', '160980', '064760', '058470', '232140', '033160', '083450', '093520', '053610', '166090', '101490', '240810', '033640', '005290', '071280', '098120', '080520', '032580', '000660', '036540', '036200', '171010', '117670', '054450', '089890', '042700', '147760', '200710', '089790', '144960', '094360', '253590', '077500', '306620', '097800', '036810', '077360', '006200', '041520', '066310', '039440', '217190', '105330', '093640', '170920', '095610', '200470', '025560', '232680', '000990', '078350', '131290',

In [21]:
import FinanceDataReader as fdr

# 네가 준 티커 리스트
target_tickers = filtered_tickers

# 전체 한국 상장 종목 불러오기
krx = fdr.StockListing('KRX')

# 티커-종목명 매핑
ticker_filtered_name_dict = {}

for ticker in target_tickers:
    name = krx.loc[krx['Code'] == ticker, 'Name']
    if not name.empty:
        ticker_filtered_name_dict[ticker] = name.values[0]
    else:
        ticker_filtered_name_dict[ticker] = None  # 못 찾은 경우 None 처리

# 결과 확인
print(ticker_filtered_name_dict)


{'136660': '큐엠씨', '219130': '타이거일렉', '031980': '피에스케이홀딩스', '036930': '주성엔지니어링', '092070': '디엔에프', '241790': '티이엠씨씨엔에스', '045300': '성우테크론', '089970': '브이엠', '319660': '피에스케이', '064520': '테크엘', '054620': 'APS', '096610': '알에프세미', '149010': '아이케이세미콘', '222160': 'NPX', '272110': '케이엔제이', '122640': '예스티', '101400': '엔시트론', '011930': '신성이엔지', '087600': '픽셀플러스', '054940': '엑사이엔씨', '030530': '원익홀딩스', '183300': '코미코', '227950': '엔투텍', '241770': '메카로', '102710': '이엔에프테크놀로지', '083310': '엘오티베큠', '003160': '디아이', '005935': '삼성전자우', '079370': '제우스', '059090': '미코', '082270': '젬백스', '005930': '삼성전자', '029460': '케이씨', '140070': '서플러스글로벌', '061970': 'LB세미콘', '084370': '유진테크', '067310': '하나마이크론', '264660': '씨앤지하이테크', '142210': '유니트론텍', '101160': '월덱스', '217500': '러셀', '086390': '유니테스트', '094170': '동운아나텍', '114810': '한솔아이원스', '131970': '두산테스나', '102120': '어보브반도체', '080220': '제주반도체', '052900': 'KX하이텍', '195870': '해성디에스', '160980': '싸이맥스', '064760': '티씨케이', '058470': '리노공업', '232140': '와이씨', '033160': '엠케이

In [23]:
# 네가 제공한 매핑 딕셔너리
# ticker_to_name = {
#     '136660': '큐엠씨', '219130': '타이거일렉', '031980': '피에스케이홀딩스', '036930': '주성엔지니어링',
#     '092070': '디엔에프', '241790': '티이엠씨씨엔에스', '045300': '성우테크론', '089970': '브이엠',
#     '319660': '피에스케이', '064520': '테크엘', '054620': 'APS', '096610': '알에프세미',
#     '149010': '아이케이세미콘', '222160': 'NPX', '272110': '케이엔제이', '122640': '예스티',
#     '101400': '엔시트론', '011930': '신성이엔지', '087600': '픽셀플러스', '054940': '엑사이엔씨',
#     '030530': '원익홀딩스', '183300': '코미코', '227950': '엔투텍', '241770': '메카로',
#     '102710': '이엔에프테크놀로지', '083310': '엘오티베큠', '003160': '디아이', '005935': '삼성전자우',
#     '079370': '제우스', '059090': '미코', '082270': '젬백스', '005930': '삼성전자', '029460': '케이씨',
#     '140070': '서플러스글로벌', '061970': 'LB세미콘', '084370': '유진테크', '067310': '하나마이크론',
#     '264660': '씨앤지하이테크', '142210': '유니트론텍', '101160': '월덱스', '217500': '러셀',
#     '086390': '유니테스트', '094170': '동운아나텍', '114810': '한솔아이원스', '131970': '두산테스나',
#     '102120': '어보브반도체', '080220': '제주반도체', '052900': 'KX하이텍', '195870': '해성디에스',
#     '160980': '싸이맥스', '064760': '티씨케이', '058470': '리노공업', '232140': '와이씨',
#     '033160': '엠케이전자', '083450': 'GST', '093520': '매커스', '053610': '프로텍',
#     '166090': '하나머티리얼즈', '101490': '에스앤에스텍', '240810': '원익IPS', '033640': '네패스',
#     '005290': '동진쎄미켐', '071280': '로체시스템즈', '098120': '마이크로컨텍솔', '080520': '오디텍',
#     '032580': '피델릭스', '000660': 'SK하이닉스', '036540': 'SFA반도체', '036200': '유니셈',
#     '171010': '램테크놀러지', '117670': '알파홀딩스', '054450': '텔레칩스', '089890': '코세스',
#     '042700': '한미반도체', '147760': '피엠티', '200710': '에이디테크놀로지', '089790': '제이티',
#     '144960': '뉴파워프라즈마', '094360': '칩스앤미디어', '253590': '네오셈', '077500': '유니퀘스트',
#     '306620': '네온테크', '097800': '윈팩', '036810': '에프에스티', '077360': '덕산하이메탈',
#     '006200': '한국전자홀딩스', '041520': '이엘씨', '066310': '큐에스아이', '039440': '에스티아이',
#     '217190': '제너셈', '105330': '케이엔더블유', '093640': '케이알엠', '170920': '엘티씨',
#     '095610': '테스', '200470': '에이팩트', '025560': '미래산업', '232680': '라온테크',
#     '000990': 'DB하이텍', '078350': '한양디지텍', '131290': '티에스이', '119830': '아이텍',
#     '074600': '원익QnC', '095340': 'ISC', '039030': '이오테크닉스', '089030': '테크윙',
#     '080580': '오킨스전자', '064290': '인텍플러스', '092220': 'KEC', '323350': '다원넥스뷰',
#     '059120': '아진엑스텍', '281820': '케이씨텍', '052860': '아이앤씨', '024850': 'HLB이노베이션',
#     '033170': '시그네틱스', '092870': '엑시콘', '317330': '덕산테코피아', '303030': '지니틱스'
# }

# 'Date' 열을 제외한 나머지 열 이름만 바꿔야 하므로
columns_renamed = {}

for col in stocks_selected.columns:
    if col == 'Date':
        columns_renamed[col] = 'Date'
    else:
        columns_renamed[col] = ticker_filtered_name_dict.get(col, col)  # 매칭 안 되면 기존 col 이름 유지

# 컬럼명 변경
stocks_selected.rename(columns=columns_renamed, inplace=True)

# 결과 확인
print(stocks_selected.head())


         Date   큐엠씨  타이거일렉  피에스케이홀딩스  주성엔지니어링   디엔에프  티이엠씨씨엔에스  성우테크론    브이엠  \
0  2020-12-30  5040  15000      8300     8210  21200     18100   4401  19900   
1  2021-12-30  3200  20900     17900    21000  22850     16600   5129  22150   
2  2022-12-29  2800  20050      6850    10600  13200     13300   3760  10600   
3  2023-12-28  1300  19990     26950    34200  23950     12860   3993  14100   
4  2024-12-30  1399  12780     38050    29450   8820      4630   2520   6870   

   피에스케이  ...   KEC  다원넥스뷰  아진엑스텍   케이씨텍  아이앤씨  HLB이노베이션  시그네틱스    엑시콘  \
0  20275  ...  2730  19250   6060  30750  4300      1415    853  15786   
1  26400  ...  3451  79900  11950  24050  3970      1260   2520  14447   
2  15400  ...  1819  54000   7790  14800  2495       948   1030   8696   
3  21050  ...  1557  10000  10170  26800  2745      3120   1236  17221   
4  16500  ...   780   5000   6580  26050  1799      2510    864  10600   

   덕산테코피아  지니틱스  
0   17800  2495  
1   30850  3220  
2   13500  1340  
3 

In [24]:
stocks_selected

,Date,큐엠씨,타이거일렉,피에스케이홀딩스,주성엔지니어링,디엔에프,티이엠씨씨엔에스,성우테크론,브이엠,피에스케이,...,KEC,다원넥스뷰,아진엑스텍,케이씨텍,아이앤씨,HLB이노베이션,시그네틱스,엑시콘,덕산테코피아,지니틱스
0,2020-12-30,5040,15000,8300,8210,21200,18100,4401,19900,20275,...,2730,19250,6060,30750,4300,1415,853,15786,17800,2495
1,2021-12-30,3200,20900,17900,21000,22850,16600,5129,22150,26400,...,3451,79900,11950,24050,3970,1260,2520,14447,30850,3220
2,2022-12-29,2800,20050,6850,10600,13200,13300,3760,10600,15400,...,1819,54000,7790,14800,2495,948,1030,8696,13500,1340
3,2023-12-28,1300,19990,26950,34200,23950,12860,3993,14100,21050,...,1557,10000,10170,26800,2745,3120,1236,17221,18690,1594
4,2024-12-30,1399,12780,38050,29450,8820,4630,2520,6870,16500,...,780,5000,6580,26050,1799,2510,864,10600,29650,1125


In [27]:
# 'Date' 열 제외한 나머지 열을 float로 변환
cols_to_convert = [col for col in stocks_selected.columns if col != 'Date']
stocks_selected[cols_to_convert] = stocks_selected[cols_to_convert].astype(float)

# 결과 확인
print(stocks_selected.dtypes)


Date         object
큐엠씨         float64
타이거일렉       float64
피에스케이홀딩스    float64
주성엔지니어링     float64
             ...   
HLB이노베이션    float64
시그네틱스       float64
엑시콘         float64
덕산테코피아      float64
지니틱스        float64
Length: 118, dtype: object


In [28]:
stocks_selected

,Date,큐엠씨,타이거일렉,피에스케이홀딩스,주성엔지니어링,디엔에프,티이엠씨씨엔에스,성우테크론,브이엠,피에스케이,...,KEC,다원넥스뷰,아진엑스텍,케이씨텍,아이앤씨,HLB이노베이션,시그네틱스,엑시콘,덕산테코피아,지니틱스
0,2020-12-30,5040.0,15000.0,8300.0,8210.0,21200.0,18100.0,4401.0,19900.0,20275.0,...,2730.0,19250.0,6060.0,30750.0,4300.0,1415.0,853.0,15786.0,17800.0,2495.0
1,2021-12-30,3200.0,20900.0,17900.0,21000.0,22850.0,16600.0,5129.0,22150.0,26400.0,...,3451.0,79900.0,11950.0,24050.0,3970.0,1260.0,2520.0,14447.0,30850.0,3220.0
2,2022-12-29,2800.0,20050.0,6850.0,10600.0,13200.0,13300.0,3760.0,10600.0,15400.0,...,1819.0,54000.0,7790.0,14800.0,2495.0,948.0,1030.0,8696.0,13500.0,1340.0
3,2023-12-28,1300.0,19990.0,26950.0,34200.0,23950.0,12860.0,3993.0,14100.0,21050.0,...,1557.0,10000.0,10170.0,26800.0,2745.0,3120.0,1236.0,17221.0,18690.0,1594.0
4,2024-12-30,1399.0,12780.0,38050.0,29450.0,8820.0,4630.0,2520.0,6870.0,16500.0,...,780.0,5000.0,6580.0,26050.0,1799.0,2510.0,864.0,10600.0,29650.0,1125.0


In [29]:
stocks_selected.to_csv('stocks_year_end.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Columns: 118 entries, Date to 지니틱스
dtypes: int64(117), object(1)
memory usage: 4.7+ KB
